# Generate Polylla mesh

In [1]:
from mesh import TetrahedronMesh
from PolyllaEdge import PolyllaEdge
from PolyllaFace import PolyllaFace

numVertices = 500

# Semiuniform Polylla

In [2]:
filename = "data/"+ str(numVertices) + "semiuniform.1"
outputname = "data/"+ str(numVertices) + "semiuniform"

node_file = filename + ".node"
ele_file = filename + ".ele"
face_file = filename + ".face"
edge_file = filename + ".edge"

mesh = TetrahedronMesh(node_file, face_file, ele_file, edge_file)

polyllaEdge_mesh = PolyllaEdge(mesh)
polyllaFace_mesh = PolyllaFace(mesh)

mesh.get_info()
polyllaEdge_mesh.get_info()
polyllaFace_mesh.get_info()


reading node file: data/500semiuniform.1.node
reading face file: data/500semiuniform.1.face
reading ele file: data/500semiuniform.1.ele
reading edge file: data/500semiuniform.1.edge
Edge: 2585 count: 4
Polyhedron [1428, 2049] has a hanging tetrahedron
Te: [1428, -1, 2049, -1] pos_origin: 1 curr: 2
Te: [1428, -1, 2049, -1] pos_origin: 1 curr: 3
Te: [1428, -1, 2049, -1] pos_origin: 1 curr: 1
New polyhedrons: [[2049], [1428]]
Edge: 2837 count: 4
Polyhedron [1884, 1889] has a hanging tetrahedron
Te: [-1, 1884, -1, -1, 1889] pos_origin: 0 curr: 1
Te: [-1, 1884, -1, -1, 1889] pos_origin: 0 curr: 2
Te: [-1, 1884, -1, -1, 1889] pos_origin: 0 curr: 0
New polyhedrons: [[1884], [1889]]
Edge: 775 count: 4
Polyhedron [161, 2097] has a hanging tetrahedron
Te: [161, -1, -1, 2097, -1] pos_origin: 1 curr: 2
Te: [161, -1, -1, 2097, -1] pos_origin: 1 curr: 4
Te: [161, -1, -1, 2097, -1] pos_origin: 1 curr: 1
New polyhedrons: [[2097], [161]]
Edge: 2114 count: 4
Polyhedron [1619, 1589] has a hanging tetrahe

## Plot Polylla mesh

In [7]:
import numpy as np
import pyvista as pv

polyhedral_mesh = polyllaFace_mesh.polyhedral_mesh

list_of_cells = []

face_list = polyllaFace_mesh.mesh.face_list
node_list = polyllaFace_mesh.mesh.node_list

polyhedral_list = []
for poly in polyhedral_mesh:
    nFaces = len(poly.faces)
    nElements = nFaces*(3 + 1) + 1 # 3 vertices per face + 1 the number of vértices and + 1 for the number of faces
    vtk_polyhedral = [nElements, nFaces]
    for face in poly.faces:
        vtk_polyhedral.append(3)
        v1 = face_list[face].v1
        v2 = face_list[face].v2
        v3 = face_list[face].v3
        vtk_polyhedral.extend([v1, v2, v3])
    polyhedral_list.append(vtk_polyhedral)

elements = np.hstack(polyhedral_list)
nodes = np.hstack([[node_list[i].x, node_list[i].y, node_list[i].z] for i in range(len(node_list))])
cell_type = np.empty(len(polyhedral_mesh), dtype='uint8')
cell_type[:] = 42

grid = pv.UnstructuredGrid(elements, cell_type, nodes)

# Calculate centroids of each polyhedral cell
cell_center = []
for poly in polyhedral_mesh:
    
    unique_vertices = []
    for face in poly.faces:
        v1 = face_list[face].v1
        v2 = face_list[face].v2
        v3 = face_list[face].v3
        unique_vertices.extend([v1, v2, v3])
    #remove duplicates
    unique_vertices = list(set(unique_vertices))
    centroid = [0.0, 0.0, 0.0]
    for vertex in unique_vertices:
        centroid[0] += node_list[vertex].x
        centroid[1] += node_list[vertex].y
        centroid[2] += node_list[vertex].z
    centroid[0] /= len(unique_vertices)
    centroid[1] /= len(unique_vertices)
    centroid[2] /= len(unique_vertices)
    cell_center.append(centroid.copy())

print(cell_center)
cell_center = np.vstack(cell_center)
        

# extract cells below the 0 xy plane
mask = cell_center[:, 2] < 0.5
cell_ind = mask.nonzero()[0]
subgrid = grid.extract_cells(cell_ind)
# advanced plotting
plotter = pv.Plotter(notebook=True)
plotter.background_color = 'white'
plotter.add_mesh(subgrid, 'lightgrey', lighting=True, show_edges=True)
plotter.add_mesh(pv.Cube(center=(0.5, 0.5, 0.5)), 'r', 'wireframe')
#plotter.add_legend([[' Input Mesh ', 'r'], [' Tessellated Mesh ', 'black']])
plotter.show(jupyter_backend='static')
    


[[0.320809837282537, 0.26749745858931356, 0.32977667428548196], [0.054508788585065596, 0.1739068921233296, 0.4577943002692463], [0.09199115991805105, 0.19465739758995523, 0.365838340053497], [0.15350926207641735, 0.3226061430624901, 0.23805308310778345], [0.12834153957239364, 0.35743349603779434, 0.12223430848951428], [0.2696734724238826, 0.3237900503293516, 0.24835227004581298], [0.17212888410407537, 0.3290269920641327, 0.06139968294788739], [0.16224581794095289, 0.35480104349001984, 0.07163877826895393], [0.3039149048449748, 0.30024882777875705, 0.27346474713542634], [0.040825744830417066, 0.3031505626462934, 0.0797359931063288], [0.3614601920028391, 0.43373268372503077, 0.9292130274478284], [0.47038248824354917, 0.4074623520482189, 0.48948773224590897], [0.6718043293988188, 0.9423233762332635, 0.30586676455451517], [0.6306858132533412, 0.9125705886245478, 0.2662107129511001], [0.6872756677978229, 0.9366607052098959, 0.3796702354217981], [0.15410330407157344, 0.9225911317886585, 0.79

ValueError: maximum supported dimension for an ndarray is 32, found 685